In [1]:
import os 
from dotenv import load_dotenv
import requests
from google.oauth2 import service_account
import xmltodict
import json
import pandas as pd 
from requests import request
import pytz
from bs4 import BeautifulSoup
from typing import Dict
from google.oauth2 import service_account
from google.cloud import storage

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os
import sys

from pyspark.sql.types import StructType, StructField, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType

from entsoe import EntsoeRawClient





In [2]:
#load env variables
load_dotenv('./creds/.env', verbose=True, override=True)

True

In [3]:
print(os.environ.get("PYSPARK_DRIVER_PYTHON"))

None


In [4]:


'''
----------------
INIT VARIABLES
----------------
'''

#setting up entsoe variables
security_token = os.environ.get("SECURITY_TOKEN")
ENTSOE_URL = 'https://transparency.entsoe.eu/api'

#setting up GCP variables
service_account_file = os.environ.get("SERVICE_ACCOUNT_FILE")
credentials = service_account.Credentials.from_service_account_file(
    service_account_file
)
gcs_bucket = os.environ.get("CLOUD_STORAGE_BUCKET")
print(gcs_bucket)

entsoe_analytics_1009


In [ ]:



# #setting up session
# entsoe_session = requests.Session()

# '''
# ----------------
# SETTING UP FUNCTION CALLS 
# ----------------
# '''

# #format dates
# def datetime_to_str(dtm: pd.Timestamp) -> str:
#     #convert timezone to UTC if it's exist and it's not on UTC already
#     if dtm.tzinfo is not None and dtm.tzinfo != pytz.UTC:
#         dtm = dtm.tz_convert("UTC")
#     fmt = '%Y%m%d%H%M'
#     dtm_str = dtm.strftime(fmt)
#     return dtm_str

# #basic requests
# def basic_requests(params: Dict, start:pd.Timestamp, end: pd.Timestamp) -> requests.Response:
#     #setting up time intervals start and stop
#     start_str = datetime_to_str(start)
#     end_str = datetime_to_str(end)

#     #setting up params and extending with custom parameter based
#     base_params = {
#         'securityToken': security_token,
#         'periodStart': start_str,
#         'periodEnd': end_str,
#     }
#     params.update(base_params)
    
#     #seting up sesssion
#     session = requests.Session()
#     response = session.get(url=ENTSOE_URL, params=params)

#     return response

# # upload data to GCS
# def upload_blob_to_gcs(bucket_name, contents, destination_blob_name):
#     # Upload file to bucket"""

#     # ID of GCS bucket
#     # bucket_name =

#     # the contents from memory to be uploaded to file
#     # contents =

#     # the ID of your GCS object
#     # destination_blob_name =

#     storage_client = storage.Client(credentials=credentials)
#     bucket = storage_client.bucket(bucket_name)
#     blob = bucket.blob(destination_blob_name)

#     blob.upload_from_string(contents)


# '''
# ----------------
# EXTRACTION
# ----------------
# '''
# #for test, we'll be querying Actual Generation Output per Generation Unit

# domain = '10YCZ-CEPS-----N'
# params_requests = {
#     'documentType': 'A73',
#     'processType': 'A16',
#     'in_Domain': {domain},
# }
# start=pd.Timestamp('202101011200', tz='Europe/Berlin')
# end=pd.Timestamp('202101011300', tz='Europe/Berlin')

# entsoe_data = basic_requests(params=params_requests, start=start, end=end)
# # entsoe_dict = xmltodict.parse(entsoe_data.text)
# # entsoe_json = json.dumps(entsoe_dict, indent=4)
# # print(entsoe_json)
# print(entsoe_data.text)

# '''
# ----------------
# LOAD
# ----------------
# '''
# # #upload to GCS
# # landing_filename=f"entsoe_data_{start}.json"
# # upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_json, destination_blob_name=landing_filename)

# #upload to GCS - XML
# landing_filename=f"entsoe_data_{start}_{domain}.xml"
# upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_data.text, destination_blob_name=landing_filename)



In [11]:



#setting up session
entsoe_client = EntsoeRawClient(security_token)
'''
----------------
SETTING UP FUNCTION CALLS 
----------------
'''

# upload data to GCS
def upload_blob_to_gcs(bucket_name, contents, destination_blob_name):
    # Upload file to bucket"""

    # ID of GCS bucket
    # bucket_name =

    # the contents from memory to be uploaded to file
    # contents =

    # the ID of your GCS object
    # destination_blob_name =

    storage_client = storage.Client(credentials=credentials)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(contents)


'''
----------------
EXTRACTION
----------------
'''
#for test, we'll be querying Actual Generation Output per Generation Unit

start=pd.Timestamp('202101011200', tz='Europe/Berlin')
end=pd.Timestamp('202101011300', tz='Europe/Berlin')
country_code= 'CZ'
country_code_from=''
country_code_to=''
type_marketagreement_type=''
contract_marketagreement_type=''


entsoe_data = entsoe_client.query_generation_per_plant(country_code, start, end)
entsoe_file_dict = xmltodict.parse(entsoe_data)
entsoe_json = json.dumps(entsoe_file_dict)

'''
----------------
LOAD
----------------
'''
#upload to GCS
landing_filename=f"entsoe_data_{country_code}.json"
upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_json, destination_blob_name=landing_filename)

# #upload to GCS - XML
# landing_filename=f"entsoe_data_{start}_{country_code}.xml"
# upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_data, destination_blob_name=landing_filename)



In [6]:
print(os.environ["SPARK_HOME"])
SPARK_HOME = os.environ["SPARK_HOME"]

/opt/spark-3.3.1-bin-hadoop3-scala2.13


In [7]:

# #setup sparksession for entry point
# SPARK_HOME = os.environ["SPARK_HOME"]
# spark = SparkSession.builder.appName("gcp_playground").config("spark.jars", f"{SPARK_HOME}/jars/spark-xml_2.13-0.15.0.jar").getOrCreate()



In [7]:
## COBA SPARK GCS CONNECTOR ##


# berangkat pak haji
# setup parameternya
gcs_bucket = gcs_bucket
path = f"gs://{gcs_bucket}/{landing_filename}"
# path ="/home/rafzul/projects/entsoe-pipelines/sample.xml"

#coba spark gcs connector
#setup sparksession for entry point - COBA GCS CONNECTOR
SPARK_HOME = os.environ["SPARK_HOME"]
spark = SparkSession.builder.appName("gcp_playground") \
    .config("spark.jars", f"{SPARK_HOME}/jars/gcs-connector-hadoop3-latest.jar") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", service_account_file) \
    .getOrCreate()



22/12/27 13:38:08 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.18.2 instead (on interface wlp58s0)
22/12/27 13:38:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/12/27 13:38:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
print(spark.conf.get("spark.hadoop.fs.AbstractFileSystem.gs.impl"))

google.cloud.hadoop.fs.gcs.GoogleHadoopFS


In [14]:
path = f"gs://{gcs_bucket}/{landing_filename}"
print(path)

#create dataframe from gcs
df_spark = spark.read.format("json") \
   .option("header","true") \
   .option("multiLine","true") \
   .option("inferSchema","true") \
   .load(path)
   

gs://entsoe_analytics_1009/entsoe_data_CZ.json
+--------------------+
|   GL_MarketDocument|
+--------------------+
|{urn:iec62325.351...|
+--------------------+



In [17]:
df_spark.show()
df_spark.printSchema()

+--------------------+
|   GL_MarketDocument|
+--------------------+
|{urn:iec62325.351...|
+--------------------+

root
 |-- GL_MarketDocument: struct (nullable = true)
 |    |-- @xmlns: string (nullable = true)
 |    |-- TimeSeries: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- MktPSRType: struct (nullable = true)
 |    |    |    |    |-- PowerSystemResources: struct (nullable = true)
 |    |    |    |    |    |-- mRID: struct (nullable = true)
 |    |    |    |    |    |    |-- #text: string (nullable = true)
 |    |    |    |    |    |    |-- @codingScheme: string (nullable = true)
 |    |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- psrType: string (nullable = true)
 |    |    |    |-- Period: struct (nullable = true)
 |    |    |    |    |-- Point: struct (nullable = true)
 |    |    |    |    |    |-- position: string (nullable = true)
 |    |    |    |    |    |-- quantity: string (nullable = tr

In [ ]:
#inferring schema and get the data type of each column and turn it into spark dataframe
datatype_infer = pd.DataFrame.from_dict(xml_file_dict[0], orient='index')

In [ ]:
from pyspark.sql.functions import explode, map_keys, col

In [20]:
#coba upload ke dataframe dari xml


with open("entsoe_analytics_1009/entsoe_data_simple.xml") as source_file:
    xml_file_dict = xmltodict.parse(source_file.read())
    types = [(f"{k}: {type(k)}") for k in xml_file_dict["GL_MarketDocument"]]
    print(types)
    df_spark = spark.createDataFrame([xml_file_dict["GL_MarketDocument"]])
    df_spark.printSchema()
    df_spark.show()
    for col in df_spark.dtypes:
        print(col)

["@xmlns: <class 'str'>", "mRID: <class 'str'>", "createdDateTime: <class 'str'>", "time_Period.timeInterval: <class 'str'>", "TimeSeries: <class 'str'>"]
root
 |-- @xmlns: string (nullable = true)
 |-- TimeSeries: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- createdDateTime: string (nullable = true)
 |-- mRID: string (nullable = true)
 |-- time_Period.timeInterval: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+--------------------+--------------------+--------------------+--------------------+------------------------+
|              @xmlns|          TimeSeries|     createdDateTime|                mRID|time_Period.timeInterval|
+--------------------+--------------------+--------------------+--------------------+------------------------+
|urn:iec62325.351:...|[{Period -> {time...|2022-12-23T08:05:44Z|dca3a3bd038f41818...|    {

In [ ]:
 df_spark.schema.prettyJson()

In [ ]:
#flatten nested df at every layer
from pyspark.sql.types import *
from pyspark.sql.import functions as f

def flatten_structs(nested_df):
    stack = [(), nested_df]
    columns = []
    
    while len(stack) > 0:
        parents
        
    